In [1]:
from qdrant_client import QdrantClient

client = QdrantClient(host="localhost", port=6333)

# List collections
print(client.get_collections())

# Check info collection
info = client.get_collection("chat_history")
print(info)


collections=[CollectionDescription(name='rag_knowledge'), CollectionDescription(name='chat_history')]
status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=0 points_count=2 segments_count=6 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=10000, flush_interval_sec=5, max_optimization_thr

In [9]:
from qdrant_client import QdrantClient
from pprint import pprint

client = QdrantClient(host="localhost", port=6333)

def inspect_qdrant():
    collections = client.get_collections().collections
    print(f"Found {len(collections)} collections:")
    
    for col in collections:
        print("\n" + "="*40)
        print(f"Collection name: {col.name}")
        try:
            info = client.get_collection(col.name)
            print(f"Points count: {info.points_count}")
            vectors = info.config.params.vectors
            if isinstance(vectors, dict):
                print("Vector names & sizes:")
                for name, vcfg in vectors.items():
                    print(f"  - {name}: size={vcfg.size}, distance={vcfg.distance}")
            else:
                print(f"Vector config: size={vectors.size}, distance={vectors.distance}")
        except Exception as e:
            print("Could not fetch collection info:", e)
        
        print("Sample points (up to 10):")
        try:
            resp, _ = client.scroll(col.name, limit=10, with_payload=True)
            points = getattr(resp, "result", resp)
            for p in points:
                vector_names = list(p.vector.keys()) if isinstance(p.vector, dict) else ["default"]
                print(f"  id={p.id}, vector_names={vector_names}, payload={p.payload}")
        except Exception as e:
            print("Could not fetch points:", e)

if __name__ == "__main__":
    inspect_qdrant()


Found 2 collections:

Collection name: rag_knowledge
Points count: 0
Vector config: size=384, distance=Cosine
Sample points (up to 10):

Collection name: chat_history
Points count: 10
Vector config: size=384, distance=Cosine
Sample points (up to 10):
  id=03f97d6c-786d-4475-a13d-cb0e3f61574c, vector_names=['default'], payload={'role': 'user', 'content': 'viết hàm đệ quy tính giai thừa sử dụng python', 'timestamp': 1758177391.542971}
  id=114e6a40-040d-4617-99c3-ae8a3d4dd446, vector_names=['default'], payload={'role': 'user', 'content': 'rag là gì', 'timestamp': 1758180768.0196736}
  id=28d1a196-bdf6-4ac5-b55d-47f8415367b8, vector_names=['default'], payload={'role': 'user', 'content': 'xin chào', 'timestamp': 1758177345.7822466}
  id=33e28889-7de7-4ac4-ab9b-74fb2fac8dad, vector_names=['default'], payload={'role': 'assistant', 'content': '```python\ndef factorial(n):\n  if n == 0:\n    return 1\n  else:\n    return n * factorial(n-1)\n\nprint(factorial(5))\n```\n```', 'timestamp': 175817